In [8]:
W = WeylGroup("A2",prefix="s")

[s1,s2] = W.simple_reflections()        


w0 = W.long_element()
e = W(1)

W_poset = W.bruhat_poset()


# Possible singularity sets
SIGMA = [x for x in Subsets(range(1,rank(W)+1))]    
SIGMA_reduced = [x for x in SIGMA if len(x)>1 and len(x)<rank(W)]    # Excluding semiregular, regular, and trivial cases


# Sorts a list L of elements from W by their length
def partial_sort(L):                    
    Bin = dict( (i,[]) for i in range(w0.length()+1))    
    for x in L:
        Bin[x.length()].append(x)    
    R = []
    for i in range(w0.length()+1):
        R = R+Bin[i]    
    return R
    
################################################################################
# Stabilizer (parabolic) subgroup W_Sigma and its cosets W/W_Sigma

# Minimal representative
def minimal_rep(x,Sigma):
    return x.coset_representative(Sigma, side='right')

# Longest element in W_Sigma
def w0_(Sigma):
    return minimal_rep(w0,Sigma).inverse() * w0
    
# Maximal representative
def maximal_rep(x,Sigma):
    return minimal_rep(x,Sigma)*w0_(Sigma)


# Stabilizer (parabolic) subgroup W_Sigma
def W_(Sigma):
    return W.bruhat_interval(e,w0_(Sigma))
   
# Lists/posets of minimal and maximal coset representatives
def Minimal_reps(Sigma):
    Min_reps = []
    Done = {1}.intersection({2})    # How else to define empty set???
    for x in W:
        if x not in Done:
            Min_reps.append(minimal_rep(x,Sigma))
            Done=Done.union(W.bruhat_interval(minimal_rep(x,Sigma),maximal_rep(x,Sigma)))
    return Min_reps

def Maximal_reps(Sigma):
    return [x*w0_(Sigma) for x in Minimal_reps(Sigma)]

def Minimal_reps_poset(Sigma):
    return W_poset.subposet(Minimal_reps(Sigma))
        
def Maximal_reps_poset(Sigma):
    return W_poset.subposet(Maximal_reps(Sigma))
           

# NEW: another way to do minimal representatives...
def Minimal_representatives(Sigma,side):
    return list(set([x.coset_representative(Sigma, side) for x in W]))
    

def conj(Sigma):                # subgroup conjugated by w0
    conj_Sigma = []
    for i in Sigma:
        conj_i = w0 * W.simple_reflections()[i] * w0
        for j in W.simple_reflections().keys():
            if W.simple_reflections()[j] == conj_i:
                conj_Sigma.append(j)
    return sorted(conj_Sigma)


SIGMA_enough = []                # Possible walls, up to the uniqe non-identity automorphism of the Dynkin diagram in types A and D
for Sigma in SIGMA:
    if (list(Sigma) not in SIGMA_enough) and (list(conj(Sigma)) not in SIGMA_enough):
        SIGMA_enough.append(list(Sigma))





################################################################################
# Singular BGG complexes


def Cosets_intersected(w,x,Sigma):             # Intersection of the interval [w,w0] and the coset x*W_Sigma
    intersection = []
    x = maximal_rep(x,Sigma)
    for y in W.bruhat_interval(x*w0_(Sigma),x):
        if w.bruhat_le(y): intersection.append(y)
    return intersection

# The unique maximum in Cosets_intersected(w,x,Sigma)  
def y(w,x,Sigma):
    return W_poset.subposet((Cosets_intersected(w,x,Sigma))).top()
    
    
# Mobius function mu, [Bjorner-Brenti, p. 52-53]            # Arguments are supposed to be either both in in Maximal_reps(Sigma)
def mu(w,x,Sigma):
    
    w = maximal_rep(w,Sigma)
    x = maximal_rep(x,Sigma)
    
    if len(Cosets_intersected(w,x,Sigma)) != 1:
        return 0
    return (-1)^(w.length()-x.length())
    


def BGG_complex(w,Sigma):
        
    w = maximal_rep(w,Sigma)
    
    objects = [x for x in Maximal_reps(Sigma) if w.bruhat_le(x) and len(Cosets_intersected(w,x,Sigma)) == 1 ]
    s_BGG = Maximal_reps_poset(Sigma).subposet(objects)
    return s_BGG


# good options for ploting Hasse digrams: .plot(figsize=[10,10], edge_color="grey", edge_style="dotted")


# Index set of the BGG complex
def BGG_complex_parameters(w,Sigma):
        
    w = maximal_rep(w,Sigma)
    w_length = w.length()
    
    return [x for x in Maximal_reps(Sigma) if w.bruhat_le(x) and len(Cosets_intersected(w,x,Sigma)) == 1 ]
    

# List of parameters in the BGG complex, arranged in a dictionary by length.
def BGG_complex_parameters_graded(w,Sigma):
        
    w = maximal_rep(w,Sigma)
    w_length = w.length()
    
    objects = BGG_complex_parameters(w,Sigma)
    
    s_BGG = dict((i,[]) for i in range(w0.length()-w_length+1))
    for x in objects:
        (s_BGG[x.length()-w_length]).append(x)    

    return s_BGG

  







################################################################################
# Kazhdan-Lusztig polynomials for singular blocks 
# [Irving: Singular blocks of the category O]



P.<q> = LaurentPolynomialRing(QQ)
KL = KazhdanLusztigPolynomial(W,q)



# A lot could be optimized below by using: Minimal_reps(conj(Sigma)) * w0 == Maximal_reps(Sigma)
# The following also holds: w0 * Minimal_reps(Sigma) == Maximal_reps(Sigma)

def KL_poly(y,w,Sigma):
#    y = minimal_rep(y,Sigma)                                            # y,w are supposed to be in Minimal_reps(Sigma), not maximal
#    w = minimal_rep(w,Sigma)
    return sum([ (-1)^(z.length())*KL.P(y*z,w) for z in W_(Sigma) ])        


def dimExt(i,y,w,Sigma):                                    # dim Ext^i(M_y,L_w) in O_Sigma
    Sigma = conj(Sigma)                                     # conj(Sigma), because we need to switch to antidominant setup
    y = minimal_rep(y*w0,Sigma)                               
    w = minimal_rep(w*w0,Sigma)    
    poly_dict = KL_poly(y,w,Sigma).dict()       
    j = (w.length()-y.length()-i)/2 
    if j not in poly_dict.keys():
        return 0
    return poly_dict[j]


def mult(y,w,Sigma):                    # Jordan-Holder multiplicity [M_y,L_w] in O_Sigma
    Sigma = conj(Sigma)                         # conj(Sigma), because we need to switch to antidominant setup
    y = w0 * minimal_rep(y*w0,Sigma)            # y,w are supposed to be in Minimal_reps(Sigma), not maximal
    w = w0 * minimal_rep(w*w0,Sigma)            
    return KL_poly(y,w,[])(1)                   # Sigma disappears [rving, p. 213.]


def Loewy_filt_Verma(w,Sigma):    # Loewy = radical filtration [Irving, p. 213.]
    
    w = maximal_rep(w,Sigma)
    
    conj_Sigma = conj(Sigma)
    
    composition_series = dict((i,[]) for i in range(w0.length()-w.length()+1))
    
    for x in Maximal_reps(Sigma):
        if w.bruhat_le(x):
            l_wx = x.length() - w.length()
            KL_wx = KL_poly(w0 * minimal_rep(w*w0,conj_Sigma),w0 * minimal_rep(x*w0,conj_Sigma),[]).dict()
            for i in KL_wx.keys():
                for j in range(KL_wx[i]):
                    (composition_series[l_wx-2*i]).append(x)
            
    return composition_series




def is_Kostant_L(w,Sigma):            # Via KL polynomials
    
    w = maximal_rep(w,Sigma)
    
    X = BGG_complex_parameters(w,Sigma)
    conj_Sigma = conj(Sigma)
    
    for x in Maximal_reps(Sigma):
        if w.bruhat_le(x):
            p = KL_poly(x*w0,w*w0,conj_Sigma)
            if (p not in [0,1]) or (p == 1 and x not in X) or (p == 0 and x in X):
                return False
    
    return True



def is_Kostant_II_L(w,Sigma):            # Via Ext
    
    w = maximal_rep(w,Sigma)
    
    X = BGG_complex_parameters(w,Sigma)
    l_w = w.length()
    
    for x in Maximal_reps(Sigma):
        l_x = x.length()
        if (l_x-l_w)%2 == 0 and w.bruhat_le(x):
            for i in range(l_x-l_w+1):
                d = dimExt(i,x,w,Sigma)
                if d>1: return False
                if d == 1:
                    if x not in X or i != (l_x-l_w):
                        return False
    return True
    


from collections import Counter
def symmetric_difference(pair):
    C1 = Counter(pair[0]) 
    C2 = Counter(pair[1])
    return(list((C1-C2).elements()), list((C2-C1).elements()))    

def Euler_BGG_complex(w,Sigma):        # calculates the Euler characteristic of the BGG complex in the graded Grothendieck group
    BGG = BGG_complex_parameters_graded(w,Sigma)
        
    w = maximal_rep(w,Sigma)
    
    Grothendieck = dict((i,([],[])) for i in range(w0.length()-w.length()+1))        # (depth ,(positive part, negative part)
    
    for k in BGG.keys():
        for M in BGG[k]:
            Loewy = Loewy_filt_Verma(M,Sigma)
            for d in Loewy.keys():
                for L in Loewy[d]:
                    (Grothendieck[k+d][k%2]).append(L)
    
    cancellation = Grothendieck                           # here we cancel composition factors as in the graded Grothendieck group
    for k in cancellation.keys():
        cancellation[k] = symmetric_difference(cancellation[k])
    
    
    del cancellation[0]                                   # remove the simple top, and zero components
    for k in cancellation.keys():
        if cancellation[k] == ([],[]):
            del cancellation[k] 
    
    return cancellation



#############################################################################

def non_Kostant_latex(s):                 # Print non-Kostant modules in a latex table format, put s=0 if want to skip regular block
    non_Kostant = {}                        
    for Sigma in SIGMA_enough:
        if s != 0 or list(Sigma) != []:
            Sigma = tuple(Sigma)
            Sigma_non_Kostant = []
            for w in Maximal_reps(Sigma):
                if is_Kostant_L(w,Sigma) == False:
                    Sigma_non_Kostant.append(w)
            if Sigma_non_Kostant != []:
                Sigma_non_Kostant = partial_sort(Sigma_non_Kostant)
            
#Needs fixing, problem with $.                print ("$\{" + ', '.join([str(i) for i in Sigma]) + "\}$" + ' & ' + "\makecell[tl]{$(" + str(Sigma_non_Kostant).replace('s', '').replace('*', '').replace('[', '').replace(']', '').replace(', ',")$, $(") + ')$} \\\\' + ' \\hline')
                print
            
                non_Kostant.update({Sigma: Sigma_non_Kostant})         # saves Dictionary={(Sigma : list of non-Kostant modules in this Wall)}
    return non_Kostant